In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import math
import cv2
import os
import unicodedata
import string
import unidecode

GATHER DATA

In [6]:
path = "../plot_sums/test"

movie_names = open("{}/movie_names.txt".format(path))
movie_names = movie_names.read().split(",,,")

box_office = open("{}/box_office.txt".format(path))
box_office = box_office.read().split(",,,")

directors = open("{}/directors.txt".format(path))
directors = directors.read().split(",,,")

metascore = open("{}/metascore.txt".format(path))
metascore = metascore.read().split(",,,")

movie_runtime = open("{}/movie_runtime.txt".format(path))
movie_runtime = movie_runtime.read().split(",,,")

movieBudget = open("{}/movieBudget.txt".format(path))
movieBudget = movieBudget.read().split(",,,")

movie_genre = open("{}/movie_genre.txt".format(path))
movie_genre = movie_genre.read().split(",,,")

# ADD DIRECTOR, CAST, MPAA, IMDB RATING
# features = ["box_office", "imdb_rating", "metascore", "movie_mpaa", "movie_names", "movie_runtime", "movieBudget"]

# for i, feature in range(features):
#     name = features[i]
#     name = open("{}/{}.txt".format(path, feature))
#     feature = feature.read().split(",,,")

In [7]:
real_runtimes = []
for runtime in movie_runtime:
    real_runtimes.append(runtime.replace(" min", ""))
movie_runtime = real_runtimes
print(len(movie_runtime))

13744


In [8]:
movie_names = movie_names[:-1]
print("Movies: {}".format(len(movie_names)))
box_office = box_office[:-1]
print("Box Office: {}".format(len(box_office)))
metascore = metascore[:-1]
print("Metascore: {}".format(len(metascore)))
movie_runtime = movie_runtime[:-1]
print("Runtime: {}".format(len(movie_runtime)))
movieBudget = movieBudget[:-1]
print("Budget: {}".format(len(movieBudget)))
movie_genre = movie_genre[:-1]
print("Genre: {}".format(len(movie_genre)))

Movies: 13743
Box Office: 13743
Metascore: 13743
Runtime: 13743
Budget: 13743
Genre: 13743


In [9]:
df = pd.DataFrame({
    'name': movie_names,
    'box_office': box_office,
    'metascore': metascore,
    'runtime': movie_runtime,
    'budget': movieBudget,
    'genre': movie_genre
})

df.head(10)

,name,box_office,metascore,runtime,budget,genre
0,Star Wars: Episode VII - The Force Awakens,936662225,81,136,245000000,"Action, Adventure, Fantasy"
1,Avatar,760507625,83,162,237000000,"Action, Adventure, Fantasy"
2,Black Panther,700059566,88,134,200000000,"Action, Adventure, Sci-Fi"
3,Avengers: Infinity War,678815482,68,149,321000000,"Action, Adventure, Sci-Fi"
4,Titanic,659325379,75,194,200000000,"Drama, Romance"
5,Jurassic World,652270625,59,124,150000000,"Action, Adventure, Sci-Fi"
6,The Avengers,623357910,69,143,220000000,"Action, Adventure, Sci-Fi"
7,Star Wars: Episode VIII - The Last Jedi,620181382,85,152,null,"Action, Adventure, Fantasy"
8,Incredibles 2,608581744,80,118,200000000,"Animation, Action, Adventure"
9,The Dark Knight,534858444,84,152,185000000,"Action, Crime, Drama"


In [11]:
df.drop(df[df['budget'] == 'null'].index, inplace = True) 
df.drop(df[df['runtime'] == 'null'].index, inplace = True) 
df.drop(df[df['box_office'] == 'null'].index, inplace = True) 
df.drop(df[df['metascore'] == 'null'].index, inplace = True) 
df.drop(df[df['budget'].str.contains('£')].index, inplace = True) 
df.drop(df[df['budget'].str.contains('€')].index, inplace = True) 
df.drop(df[df['budget'].str.contains('[A-Za-z]', regex=True)].index, inplace = True) 


In [12]:
len(df)

5485

In [14]:
df

,name,box_office,metascore,runtime,budget,genre
0,Star Wars: Episode VII - The Force Awakens,936662225,81,136,245000000,"Action, Adventure, Fantasy"
1,Avatar,760507625,83,162,237000000,"Action, Adventure, Fantasy"
2,Black Panther,700059566,88,134,200000000,"Action, Adventure, Sci-Fi"
3,Avengers: Infinity War,678815482,68,149,321000000,"Action, Adventure, Sci-Fi"
4,Titanic,659325379,75,194,200000000,"Drama, Romance"
5,Jurassic World,652270625,59,124,150000000,"Action, Adventure, Sci-Fi"
6,The Avengers,623357910,69,143,220000000,"Action, Adventure, Sci-Fi"
8,Incredibles 2,608581744,80,118,200000000,"Animation, Action, Adventure"
9,The Dark Knight,534858444,84,152,185000000,"Action, Crime, Drama"
10,Rogue One,532177324,65,133,200000000,"Action, Adventure, Sci-Fi"


In [112]:
"124 min".replace(" min", "")

'124'

In [22]:
test = []
a = ["$12321391924", "DKK 123943948312", "asdfa1238213", "2343256234", "234252635", "322423424", "23425"]
for value in a:
    if bool(re.match('^[1234567890]+$', value)) == True:
        test.append(value)
    else:
        print('false')

false
false
false


In [21]:
len(test)

5485